## Benchmark 5: Structured bootstrapping
OK, so internal benchmarking strategy #2 here. Going to start calling it bootstrapping because I am using random sampling with replacement and am envisioning a bagging approach to the final model. Big takeaway from the first cross validation-like approach (really ended up being more like bootstrapping anyway...) was that to get good approximations of the public leader board score we should not be aggregating SMAPE scores across forecast origins. This seems to result in worse performance, the explanation being that we are including data from some anomalous and some nonanomalous timepoints. When, in reality a given timepoint is either an anomaly or not. This causes the naive model to do somewhat badly all the time rather than OK on nonanomalous test time points and very badly on anomalous ones.

So, the strategy here will be to structure our bootstrapping while still treating each county as an individual datapoint. To generate a sample of size n the procedure will be as follows:
1. Remove and sequester some timepoints dataset-wide for test data.
2. From the remaining, pick a random timepoint.
3. From the randomly chosen timepoint randomly choose n counties.
4. Go to step 2.

This procedure could be repeated with different test-train splits, or even test train splits on non-overlapping subsets of the data. Does that make it bootstrapping inside of cross-validation fold? I don't know - I think the specific terminology here is less important than a precise description of what we are actually doing.


1. [Abbreviations & definitions](#abbrevations_definitions)
2. [Load & inspect](#load_inspect)
3. [Build data structure](#build_data_structure)

<a name="abbreviations_definitions"></a>
### 1. Abbreviations & definitions
+ MBD: microbusiness density
+ MBC: microbusiness count
+ OLS: ordinary least squares
+ Model order: number of past timepoints used as input data for model training
+ Origin (forecast origin): last known point in the input data
+ Horizon (forecast horizon): number of future data points predicted by the model
+ SMAPE: Symmetric mean absolute percentage error

<a name="load_inspect"></a>
### 2. Load & inspect

In [1]:
# Add parent directory to path to allow import of config.py
import sys
sys.path.append('..')
import config as conf
import functions.data_manipulation_functions as data_funcs

import numpy as np
import pandas as pd
import multiprocessing as mp
from statistics import NormalDist

print(f'Python: {sys.version}')
print()
print(f'Numpy {np.__version__}')
print(f'Pandas {pd.__version__}')

Python: 3.10.0 | packaged by conda-forge | (default, Nov 20 2021, 02:24:10) [GCC 9.4.0]

Numpy 1.23.5
Pandas 1.4.3


In [2]:
# Load up training file, set dtype for first day of month
training_df = pd.read_csv(f'{conf.KAGGLE_DATA_PATH}/train.csv.zip', compression='zip')
training_df['first_day_of_month'] =  pd.to_datetime(training_df['first_day_of_month']).astype(int)
training_df.drop(['row_id', 'county','state'], axis=1, inplace=True)
training_df.head()

,cfips,first_day_of_month,microbusiness_density,active
0,1001,1564617600000000000,3.007682,1249
1,1001,1567296000000000000,2.884870,1198
2,1001,1569888000000000000,3.055843,1269
3,1001,1572566400000000000,2.993233,1243
4,1001,1575158400000000000,2.993233,1243


<a name="build_data_structure"></a>
### 1. Build data structure
First thing to do is build a data structure that will make resampling as easy as possible and persist it to disk. Then we will implement resampling. This way when experimenting with/training models, it will be easy to generate samples on they fly without having to rebuild the whole thing in memory each time.

The trick here is that in the first dimension we want the timepoints, then the counties, then the data columns. But, the 'timepoints' we want in the first dimension are really the forecast origins for a subset of the timecourse.

Maybe the best way to think about it is to forget the model order, forecast horizon and forecast origin location for now and just pick a data instance size. Then, when feeding the sample to a model we can break each block into input and forecast halves on the fly however we want. This will also solve the problem we had in notebook #07 where if model order != forecast horizon NumPy complains about ragged arrays.

Not sure if there maybe would be a speed advantage of sharding the data to multiple files here - i.e. each timepoint gets its own file and then worker processes can be assigned to sample from and train on the timepoints independently. Let's save that for a later optimization to minimize complexity out of the gate and not spend time prematurely optimizing something that ends up not being the right idea.

In [3]:
# First thing, let's give each timepoint an integer number so we don't have to 
# work with the strings/datetimes in 'first_day_of_month' directly and clean up
# columns we won't need.

training_df['timepoint'] = training_df.groupby(['cfips']).cumcount()
training_df.head()

,cfips,first_day_of_month,microbusiness_density,active,timepoint
0,1001,1564617600000000000,3.007682,1249,0
1,1001,1567296000000000000,2.884870,1198,1
2,1001,1569888000000000000,3.055843,1269,2
3,1001,1572566400000000000,2.993233,1243,3
4,1001,1575158400000000000,2.993233,1243,4


In [4]:
# Let's also add a column with difference detrended data (see notebook #02.2)

# Makes sure the rows are in chronological order within each county
training_df = training_df.sort_values(by=['cfips', 'first_day_of_month'])

# Calculate and add column for month to month change in MBD
training_df['microbusiness_density_change'] = training_df.groupby(['cfips'])['microbusiness_density'].diff() # type: ignore
training_df.dropna(inplace=True)
training_df.head()

,cfips,first_day_of_month,microbusiness_density,active,timepoint,microbusiness_density_change
1,1001,1567296000000000000,2.884870,1198,1,-0.122812
2,1001,1569888000000000000,3.055843,1269,2,0.170973
3,1001,1572566400000000000,2.993233,1243,3,-0.062610
4,1001,1575158400000000000,2.993233,1243,4,0.000000
5,1001,1577836800000000000,2.969090,1242,5,-0.024143


In [5]:
block_size = 5

# Going to need a list of unique cfips IDs to retrieve the counties
cfips_list = training_df['cfips'].drop_duplicates(keep='first').to_list()
print(f'Num counties: {len(cfips_list)}')

# The possible block left edges are the timepoints so loop on those
num_timepoints = training_df['timepoint'].nunique()
print(f'Num timepoints: {num_timepoints}\n')

timepoints = []

for left_edge in range(1, (num_timepoints - block_size)):

    # The right edge of this block is the left edge number
    # plus the blocksize
    right_edge = left_edge + block_size
    print(f'\tBlock range: {left_edge} - {right_edge}')

    # Holder for individual blocks
    blocks = []

    # Now we go through each county and get this block range
    for cfips in cfips_list:

        # Get data for just this county
        county_data = training_df[training_df['cfips'] == cfips]

        # Get rows for the block range
        county_data = county_data.loc[(county_data['timepoint'] >= left_edge) & (county_data['timepoint'] < right_edge)]

        # Convert block range rows to numpy and collect
        blocks.append(county_data.to_numpy())
    
    # Convert list of blocks to numpy and collect
    print(f'\tBlock shape: {np.array(blocks).shape}\n')

    timepoints.append(np.array(blocks))

# Convert final result to numpy
timepoints = np.array(timepoints)

Num counties: 3135
Num timepoints: 38

	Block range: 1 - 6
	Block shape: (3135, 5, 6)

	Block range: 2 - 7
	Block shape: (3135, 5, 6)

	Block range: 3 - 8
	Block shape: (3135, 5, 6)

	Block range: 4 - 9
	Block shape: (3135, 5, 6)

	Block range: 5 - 10
	Block shape: (3135, 5, 6)

	Block range: 6 - 11
	Block shape: (3135, 5, 6)

	Block range: 7 - 12
	Block shape: (3135, 5, 6)

	Block range: 8 - 13
	Block shape: (3135, 5, 6)

	Block range: 9 - 14
	Block shape: (3135, 5, 6)

	Block range: 10 - 15
	Block shape: (3135, 5, 6)

	Block range: 11 - 16
	Block shape: (3135, 5, 6)

	Block range: 12 - 17
	Block shape: (3135, 5, 6)

	Block range: 13 - 18
	Block shape: (3135, 5, 6)

	Block range: 14 - 19
	Block shape: (3135, 5, 6)

	Block range: 15 - 20
	Block shape: (3135, 5, 6)

	Block range: 16 - 21
	Block shape: (3135, 5, 6)

	Block range: 17 - 22
	Block shape: (3135, 5, 6)

	Block range: 18 - 23
	Block shape: (3135, 5, 6)

	Block range: 19 - 24
	Block shape: (3135, 5, 6)

	Block range: 20 - 25
	B

In [6]:
print(f'Timepoints shape: {timepoints.shape}')
print()
print('Column types:')

for column in timepoints[0,0,0,0:]:
    print(f'\t{type(column)}')

print()
print(f'Example block:\n{timepoints[0,0,0:,]}')

Timepoints shape: (32, 3135, 5, 6)

Column types:
	<class 'numpy.float64'>
	<class 'numpy.float64'>
	<class 'numpy.float64'>
	<class 'numpy.float64'>
	<class 'numpy.float64'>
	<class 'numpy.float64'>

Example block:
[[ 1.0010000e+03  1.5672960e+18  2.8848701e+00  1.1980000e+03
   1.0000000e+00 -1.2281170e-01]
 [ 1.0010000e+03  1.5698880e+18  3.0558431e+00  1.2690000e+03
   2.0000000e+00  1.7097300e-01]
 [ 1.0010000e+03  1.5725664e+18  2.9932332e+00  1.2430000e+03
   3.0000000e+00 -6.2609900e-02]
 [ 1.0010000e+03  1.5751584e+18  2.9932332e+00  1.2430000e+03
   4.0000000e+00  0.0000000e+00]
 [ 1.0010000e+03  1.5778368e+18  2.9690900e+00  1.2420000e+03
   5.0000000e+00 -2.4143200e-02]]


OK, looks good. We could use int for some of these columns, but we need float for the MBD, so let's leave it alone rather than using mixed types in a NumPy array. So, that's it - easy. Let's round trip it and try recovering some our values back into a nice pandas dataframe as a sanity check.

In [7]:
# Write to disk
output_file = f'{conf.DATA_PATH}/parsed_data/structured_bootstrap_blocksize{block_size}.npy'
np.save(output_file, timepoints)

# Check round-trip
loaded_timepoints = np.load(output_file)

# Inspect
print(f'Timepoints shape: {loaded_timepoints.shape}')
print(f'Example block:\n{loaded_timepoints[0,0,0:,]}')

Timepoints shape: (32, 3135, 5, 6)
Example block:
[[ 1.0010000e+03  1.5672960e+18  2.8848701e+00  1.1980000e+03
   1.0000000e+00 -1.2281170e-01]
 [ 1.0010000e+03  1.5698880e+18  3.0558431e+00  1.2690000e+03
   2.0000000e+00  1.7097300e-01]
 [ 1.0010000e+03  1.5725664e+18  2.9932332e+00  1.2430000e+03
   3.0000000e+00 -6.2609900e-02]
 [ 1.0010000e+03  1.5751584e+18  2.9932332e+00  1.2430000e+03
   4.0000000e+00  0.0000000e+00]
 [ 1.0010000e+03  1.5778368e+18  2.9690900e+00  1.2420000e+03
   5.0000000e+00 -2.4143200e-02]]


OK, not surprisingly - we got the same thing back. Last thing to do before we call this done is to test if we can get our dates, row_ids and cfips back into a format that matches the original data.

In [8]:
# Grab an example date column
test_dates = loaded_timepoints[0,0,0:,1] # type: ignore
print(f'Test dates: {test_dates}\ndtype: {type(test_dates)}\nelement dtype: {type(test_dates[0])}\n')

# Cast float64 to int64
test_dates = test_dates.astype(np.int64)
print(f'Test dates: {test_dates}\ndtype: {type(test_dates)}\nelement dtype: {type(test_dates[0])}\n')

# Convert to pandas dataframe with dtype datetime64[ns] and column name 'first_day_of_month'
test_dates_df = pd.DataFrame(pd.to_datetime(test_dates), columns=['first_day_of_month']).astype('datetime64')
test_dates_df.info()

Test dates: [1.5672960e+18 1.5698880e+18 1.5725664e+18 1.5751584e+18 1.5778368e+18]
dtype: <class 'numpy.ndarray'>
element dtype: <class 'numpy.float64'>

Test dates: [1567296000000000000 1569888000000000000 1572566400000000000
 1575158400000000000 1577836800000000000]
dtype: <class 'numpy.ndarray'>
element dtype: <class 'numpy.int64'>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   first_day_of_month  5 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 168.0 bytes


In [9]:
test_dates_df.head()

,first_day_of_month
0,2019-09-01
1,2019-10-01
2,2019-11-01
3,2019-12-01
4,2020-01-01


In [10]:
# Grab an example cfips column
test_cfips = loaded_timepoints[0,0,0:,0] # type: ignore
print(f'Test cfips: {test_cfips}\ndtype: {type(test_cfips)}\nelement dtype: {type(test_cfips[0])}\n')

# Cast float64 to int64
test_cfips = test_cfips.astype(np.int64)
print(f'Test cfips: {test_cfips}\ndtype: {type(test_cfips)}\nelement dtype: {type(test_cfips[0])}\n')

# Convert to pandas dataframe with dtype int64 and column name 'cfips'
test_cfips_df = pd.DataFrame(test_cfips, columns=['cfips']).astype('int64')
test_cfips_df.info()

Test cfips: [1001. 1001. 1001. 1001. 1001.]
dtype: <class 'numpy.ndarray'>
element dtype: <class 'numpy.float64'>

Test cfips: [1001 1001 1001 1001 1001]
dtype: <class 'numpy.ndarray'>
element dtype: <class 'numpy.int64'>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   cfips   5 non-null      int64
dtypes: int64(1)
memory usage: 168.0 bytes


In [11]:
test_cfips_df.head()

,cfips
0,1001
1,1001
2,1001
3,1001
4,1001


Ok, happy - making the rwo ID is just a string join from here, so no problem. An if for some reason we want the string county or state back, we can use a CFIPS lookup table. Time to move on.